In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches, lines
from pathlib import Path
import numpy as np
import math
import re

In [ ]:
plt.rcParams["font.size"] = "10"
plt.rcParams["axes.labelsize"] = "10"
plt.rcParams["xtick.labelsize"] = "10"
plt.rcParams["ytick.labelsize"] = "10"

In [ ]:
data_path = Path("../data")
data_path_C432_XNH = data_path / "C432_XNH_nuclei"
data_path_Y391_XNH = data_path / "Y391_XNH_nuclei"
data_path_Y489_XNH = data_path / "Y489_XNH_nuclei"

plots_path = Path("../plots")

In [ ]:
get_volume_per_vx = lambda scale: scale[0] * scale[1] * scale[2] / 1e9

scale_C432_XNH_mag2 = np.array([100, 100, 100]) * 2
scale_Y391_XNH_mag2 = np.array([100, 100, 100]) * 2
scale_Y489_XNH_mag2 = np.array([100, 100, 100]) * 2

volume_per_vx_C432_XNH_mag2 = get_volume_per_vx(scale_C432_XNH_mag2)
volume_per_vx_Y391_XNH_mag2 = get_volume_per_vx(scale_Y391_XNH_mag2)
volume_per_vx_Y489_XNH_mag2 = get_volume_per_vx(scale_Y489_XNH_mag2)

get_diameter_for_volume = lambda volume: 2 * pow(3 * volume / (4 * math.pi), 1/3)

# Volume-sphericity plots

In [ ]:
data_all_C432_XNH = data_path_C432_XNH / "C432_XNH-v10-segv1-masked_nuclei_data_sphericity_vx.csv.gz"
data_true_positives_C432_XNH = data_path_C432_XNH / "C432_XNH-eval-v10-segv1-18boxes_nuclei_true_positives_volumes_sphericities_vx.csv.gz"
data_false_positives_C432_XNH = data_path_C432_XNH / "C432_XNH-eval-v10-segv1-18boxes_nuclei_false_positives_volumes_sphericities_vx.csv.gz"
data_thresholds_C432_XNH = data_path_C432_XNH / "C432_XNH-v10-segv1_thresholds_volume_sphericity.csv"

data_all_Y391_XNH = data_path_Y391_XNH / "Y391_XNH-v10-segv1-masked_nuclei_data_sphericity_vx.csv.gz"
data_true_positives_Y391_XNH = data_path_Y391_XNH / "Y391_XNH-eval-v10-segv1_nuclei_true_positives_volumes_sphericities_vx.csv.gz"
data_false_positives_Y391_XNH = data_path_Y391_XNH / "Y391_XNH-eval-v10-segv1_nuclei_false_positives_volumes_sphericities_vx.csv.gz"
data_thresholds_Y391_XNH = data_path_Y391_XNH / "Y391_XNH-v10-segv1_thresholds_volume_sphericity.csv"

data_all_Y489_XNH = data_path_Y489_XNH / "Y489_ID16A_v18-v13-29boxes-segv1-volume-masked-v2_nuclei_data_sphericity_vx.csv.gz"
data_true_positives_Y489_XNH = data_path_Y489_XNH / "Y489_ID16A_v18-eval-v13-29boxes-segv1-testv3_nuclei_true_positives_volumes_sphericities_vx.csv.gz"
data_false_positives_Y489_XNH = data_path_Y489_XNH / "Y489_ID16A_v18-eval-v13-29boxes-segv1-testv3_nuclei_false_positives_volumes_sphericities_vx.csv.gz"
data_thresholds_Y489_XNH = data_path_Y489_XNH / "Y489_ID16A_v18-v13_thresholds_volume_sphericity.csv"

In [ ]:
def scatter_volume_sphericity(ax, volume_per_vx, data_all_path, data_true_positives_path, data_false_positives_path, thresholds_path):
    detected_nuclei = pd.read_csv(data_all_path, compression="gzip")

    segment_volumes = detected_nuclei["volumes"] * volume_per_vx
    sphericities = detected_nuclei["sphericities"]
    ax.set(xlabel=r"volume in µm$^3$", ylabel="sphericity", xscale="log", ylim=(0,3), xlim=(1E-2, 1.E7),xticks=[1.E-2, 1.E0, 1.E2, 1.E4, 1.E6,], yticks=np.arange(0, 4, 1))

    # If the points are not rasterized, the resulting image file will become very large
    print(segment_volumes.max())
    ax.scatter(segment_volumes, sphericities, color="cyan", label="all", s=5, alpha=0.5, rasterized=True)
   
    nuclei_true_positives = pd.read_csv(data_true_positives_path, compression="gzip")
    nuclei_false_positives = pd.read_csv(data_false_positives_path, compression="gzip")

    eval_tp_volumes = nuclei_true_positives["volumes"] * volume_per_vx
    eval_tp_sphericities = nuclei_true_positives["sphericities"]
        
    eval_fp_volumes = nuclei_false_positives["volumes"] * volume_per_vx
    eval_fp_sphericities = nuclei_false_positives["sphericities"]
                
    ax.scatter(eval_tp_volumes, eval_tp_sphericities, color="blue", label="true pos.", s=5, alpha=0.8, rasterized=True)
    ax.scatter(eval_fp_volumes, eval_fp_sphericities, color="orange", label="false pos.", s=5, alpha=0.8, rasterized=True)
    ax.legend()
        
    thresholds_df = pd.read_csv(thresholds_path)
    threshold_volume = thresholds_df["threshold_volume"][0] * volume_per_vx
    ax.axvline(threshold_volume, color="black", linestyle="dashdot", linewidth=1.5)
    ax.annotate("volume thr.", (threshold_volume + threshold_volume/10, 1.2), rotation=90)

    threshold_sphericity = thresholds_df["threshold_sphericity"][0]
    ax.axhline(threshold_sphericity, color="black", linestyle="dashdot", linewidth=1.5)
    ax.annotate("spher. thr.", (1.E4, threshold_sphericity - 0.15))

    ax.axhline(1, color="dimgray", linestyle="dashdot", linewidth=1.5)
    ax.annotate("spher. = 1", (1.E4, 0.85), color="dimgray")
    ax.legend(facecolor="None")
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

In [ ]:
fig = plt.figure(figsize=(3, 3))
ax = fig.add_subplot()
scatter_volume_sphericity(ax, volume_per_vx_C432_XNH_mag2, data_all_C432_XNH, data_true_positives_C432_XNH, data_false_positives_C432_XNH, data_thresholds_C432_XNH)
plt.savefig(plots_path/"F3e_volumes_sphericities_thresholded_C432_XNH.svg", format="svg", dpi=300, bbox_inches="tight")

In [ ]:
fig = plt.figure(figsize=(3, 3))
ax = fig.add_subplot()
scatter_volume_sphericity(ax, volume_per_vx_Y391_XNH_mag2, data_all_Y391_XNH, data_true_positives_Y391_XNH, data_false_positives_Y391_XNH, data_thresholds_Y391_XNH)
plt.savefig(plots_path/"F3k_volumes_sphericities_thresholded_Y391_XNH.svg", format="svg", dpi=300, bbox_inches="tight")

In [ ]:
fig = plt.figure(figsize=(3, 3))
ax = fig.add_subplot()
scatter_volume_sphericity(ax, volume_per_vx_Y489_XNH_mag2, data_all_Y489_XNH, data_true_positives_Y489_XNH, data_false_positives_Y489_XNH, data_thresholds_Y489_XNH)
plt.savefig(plots_path/"F3q_volumes_sphericities_thresholded_Y489_XNH.svg", format="svg", dpi=300, bbox_inches="tight")

# Metrics per bounding box plots

In [ ]:
bbox_metrics_crossvalidation_C432_XNH = pd.read_csv(data_path_C432_XNH / "C432_XNH-v10-segv1_metrics_volume_sphericity_threshold_cross_validation_boxes.csv")
bbox_metrics_test_C432_XNH = pd.read_csv(data_path_C432_XNH / "C432_XNH-v10-segv1_metrics_volume_sphericity_threshold_test_boxes.csv")

bbox_metrics_crossvalidation_Y391_XNH = pd.read_csv(data_path_Y391_XNH / "Y391_XNH-v10-segv1_metrics_volume_sphericity_threshold_cross_validation_boxes.csv")
bbox_metrics_test_Y391_XNH = pd.read_csv(data_path_Y391_XNH / "Y391_XNH-v10-segv1_metrics_volume_sphericity_threshold_test_boxes.csv")

bbox_metrics_Y489_XNH = pd.read_csv(data_path_Y489_XNH / "Y489_ID16A_v18_eval_v13_29boxes_segv1_testv3_metrics_volume_sphericity_threshold_test_boxes.csv")
test_box_names_Y489_XNH = ["GL_border2", "MCL_border1", "GL_ctr3", "EPL_ctr1", "MCL_ctr2", "EPL_border3"]
bbox_metrics_classifier_selection_Y489_XNH = bbox_metrics_Y489_XNH[~bbox_metrics_Y489_XNH["box_name"].isin(test_box_names_Y489_XNH)]
bbox_metrics_test_Y489_XNH = bbox_metrics_Y489_XNH[bbox_metrics_Y489_XNH["box_name"].isin(test_box_names_Y489_XNH)]

In [ ]:
def scatter_precision_recall_per_box(ax, metrics_per_bbox_trainval, metrics_per_bbox_test, has_dark, trainval_label = "cross-val."):
    colors = {"EPL": "#206Cffff","MCL": "#595959ff", "GL": "#FF500Fff"}
    region_markers = {"ctr": "s", "border": "^", "dark": "v"} if has_dark else {"ctr": "s", "border": "^"}

    def get_color_and_marker(box_name):
        layer = re.findall("EPL|MCL|GL", box_name)[0]
        region = re.findall("ctr|border|dark", box_name)[0]
        return colors[layer], region_markers[region]
    
    box_name_without_number = lambda box_name: box_name[:-1] if box_name[-1].isnumeric() else box_name

    for bbox_metrics_entry in metrics_per_bbox_trainval.itertuples():
        box_name = bbox_metrics_entry.box_name
        precision = bbox_metrics_entry.precision
        recall = bbox_metrics_entry.recall_thresholding_and_classifier
        color, marker = get_color_and_marker(box_name_without_number(box_name))
        ax.scatter(precision, recall, color=color, marker=marker, s=50, facecolors="None")
    
    for bbox_metrics_entry in metrics_per_bbox_test.itertuples():
        box_name = bbox_metrics_entry.box_name
        precision = bbox_metrics_entry.precision
        recall = bbox_metrics_entry.recall_thresholding_and_classifier
        color, marker = get_color_and_marker(box_name_without_number(box_name))
        ax.scatter(precision, recall, color="black", marker=marker, s=50, facecolors=color)
        
    ax.set(xticks=np.arange(0, 1.1, 0.25), xlabel="precision", xlim=(0, 1.1),
           yticks=np.arange(0, 1.1, 0.25), ylabel="recall", ylim=(0, 1.1))
    
    patch_crossval = patches.Patch(label=trainval_label, edgecolor="black", facecolor="None", linewidth=1, linestyle="-")
    patch_test = patches.Patch(color="black", label="test")
    patches_splits = [patch_crossval, patch_test]
    markers_regions = [lines.Line2D([], [], marker=marker, label=region, linestyle="none", color="black") for region, marker in region_markers.items()]
    patches_layers = [patches.Patch(color=color, label=layer) for layer, color in colors.items()]
    ax.legend(loc="upper left", handles=markers_regions + patches_splits + patches_layers, facecolor="None")

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

In [ ]:
fig = plt.figure(figsize=(3, 3))
ax = fig.add_subplot()
scatter_precision_recall_per_box(ax, bbox_metrics_crossvalidation_C432_XNH, bbox_metrics_test_C432_XNH, has_dark=False)
plt.savefig(plots_path/"F3f_v3_metrics_boxes_thresholded_C432_XNH.svg", format="svg", bbox_inches="tight")

In [ ]:
fig = plt.figure(figsize=(3, 3))
ax = fig.add_subplot()
scatter_precision_recall_per_box(ax, bbox_metrics_crossvalidation_Y391_XNH, bbox_metrics_test_Y391_XNH, has_dark=False)
plt.savefig(plots_path/"F3l_v3_metrics_boxes_thresholded_Y391_XNH.svg", format="svg", bbox_inches="tight")

In [ ]:
fig = plt.figure(figsize=(3, 3))
ax = fig.add_subplot()
scatter_precision_recall_per_box(ax, bbox_metrics_classifier_selection_Y489_XNH, bbox_metrics_test_Y489_XNH, has_dark=True, trainval_label="class.sel.")
plt.savefig(plots_path/"F3r_v3_metrics_boxes_thresholded_Y489_XNH.svg", format="svg", bbox_inches="tight")

# Plots to compare training data amounts for Y489_XNH

In [ ]:
scores_path = data_path_Y489_XNH / "compare_Y489_train_data_amount.csv"

filtered_nuclei_paths_v13 = {}

filtered_nuclei_path_v11 = data_path_Y489_XNH / "Y489_ID16A_v18-v11-segv1-volume-masked-v2_nuclei_data_sphericity_vx_filtered_nuclei_data_sphericity_vx.csv.gz"

for i in range(5, 30, 6): 
    filtered_nuclei_paths_v13[i] = data_path_Y489_XNH / f"Y489_ID16A_v18-v13-{i}boxes-segv1-volume-masked-v2_nuclei_data_sphericity_vx_filtered_nuclei_data_sphericity_vx.csv.gz"

In [ ]:
def plot_train_data_scores(path, ax):
    data = pd.read_csv(path)
    for i, row in data.iterrows():
        if row["contains_C432_train_data"]:
            data.at[i, "contains_C432_train_data"] = 30
    data["sum_bboxes"] = data["bboxes_Y489"] + data["contains_C432_train_data"]
    data["percentage_Y489"] = 1 - (data["contains_C432_train_data"] / data["sum_bboxes"])
    x_lables = ["0+29", "30+0", "30+5", "30+11", "30+17", "30+23", "30+29"]
    data = data.sort_values("sum_bboxes")

    color_blend = data["percentage_Y489"].apply(lambda x: (0, 1-x, x))

    for i, score in enumerate(["precision", "recall", "f1"]):
        ax[i].set(ylim=(0.795, 1.005), xlim=(25, 65), ylabel=score)
        
        ax[i].set(xticks=[30, 40, 50, 60])
        ax[i].set(yticks=[0.8, 0.9, 1.0])
        ax[i].tick_params(axis='x', rotation=60)
        ax[i].scatter(data["sum_bboxes"], data[score], color=color_blend)
        for j, txt in enumerate(x_lables):
            ax[i].annotate(txt, (data["sum_bboxes"].iloc[j] + 0.5 , data[score].iloc[j] - 0.005), fontsize=14, rotation=60)


In [ ]:
def plot_absolute_numbers_filtered(ax, filtered_v13_paths):
    data_v13 = {i: pd.read_csv(path, compression="gzip") for i, path in filtered_v13_paths.items()}

    region_sum = 0
    diff_avg_nuclei_dict = {}
    for i, data in data_v13.items():
        num_nuclei = data.shape[0]
        region_sum += num_nuclei
        diff_avg_nuclei_dict[i] = num_nuclei
    #divide all values by region_sum
    diff_avg_nuclei_dict = {k: v-(region_sum/len(data_v13)) for k, v in diff_avg_nuclei_dict.items()}
    ax.bar(diff_avg_nuclei_dict.keys(), diff_avg_nuclei_dict.values())
    ax.set(xticks=[5, 11, 17, 23, 29])
    ax.set(xlabel="Number of added Y489 training bboxes", ylabel=f"difference to average num of nuclei")
    ax.axhline(y=0, color='dimgray', linestyle='dashed')
    ax.text(0.6, 0.6, f"average: {round(region_sum/len(data_v13))}", transform=ax.transAxes, color='dimgray')
   


In [ ]:
fig = plt.figure(figsize=(2,2))
ax = fig.add_subplot()
plot_absolute_numbers_filtered(ax, filtered_nuclei_paths_v13)
plt.savefig(plots_path/"F3c_compare_absolute_difference.svg", format="svg")

In [ ]:

plt.rcParams["font.size"] = "24"
plt.rcParams["axes.labelsize"] = "24"
plt.rcParams["xtick.labelsize"] = "24"
plt.rcParams["ytick.labelsize"] = "24"

ncols = 3
nrows = 1
subplot_size = 4
fig, ax = plt.subplots(nrows, ncols, figsize=(ncols * subplot_size, nrows * subplot_size), constrained_layout=True)
fig.supxlabel("Number of train bboxes C432 + Y489")
plot_train_data_scores(scores_path, ax)
plt.savefig(plots_path/"F3c_compare_added_training_data.svg", format="svg")